# paid & banner & organic 그룹별 유저의 비교견적 신청율 비교

In [1]:
# ✅ ipynb template !!!✅
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency, beta
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.multitest import multipletests
from IPython.display import display, Markdown

# pandas 출력 설정
pd.set_option('display.max_columns', None)      # 모든 열을 출력
pd.set_option('display.width', 2000)            # 한 줄에 최대 출력 길이 설정 (충분히 크게)
pd.set_option('display.max_colwidth', None)     # 각 열의 최대 출력 길이 제한 해제

# ✅ 전체 float 출력 포맷 소수점 4자리로 고정 (현재 세션에만 적용. python 다시 시작하면 다시 설정해줘야 함.)
pd.set_option('display.float_format', '{:.4f}'.format)

# ❄️ snowflake 설정 1
from sqlalchemy import create_engine
user = 'namgonlee_svc'
account = 'MAESGVY-TY96489'
private_key_file = '/Users/encr25n10048/Documents/namgonlee_svc_key.p8'
warehouse = 'USER_XSMALL'       # 필요 시 수정
# database = 'ENCAR'            # 필요 시 수정
# schema = 'LOGS_MART'          # 필요 시 수정
# role = 'USER_XSMALL'          # 필요 시 수정

# ❄️ snowflake 설정 2
# 🔐 private key 로드 및 변환
from cryptography.hazmat.primitives import serialization
with open(private_key_file, "rb") as key_file:
    p_key = serialization.load_pem_private_key(
        key_file.read(),
        password=None,
    )
pkb = p_key.private_bytes(
    encoding=serialization.Encoding.DER,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption()
)

# ❄️ snowflake 설정 3
# SQLAlchemy 엔진 생성
engine = create_engine(
    f'snowflake://{user}@{account}?warehouse={warehouse}&',
    connect_args={
        'private_key': pkb,
    }
)

# dataframe output 에 대해 복사
# df.to_clipboard(index=False, sep='\t')

In [3]:
# sql 문 안에 변수처리하면 작동하지 않기 때문에 아래와 python 에서 변수처리 한 뒤 sql에는 파이썬 변수를 넣어야 한다.
# 변수list
# 변수 사용시 -> '{변수명}'               

# abtestId = 'ab_685cd44042161e1e5fb4245c'
start_date = '2025-07-22'
end_date = '2025-07-29'


# SQL 실행 및 결과 불러오기
query = f'''
WITH

user_master AS (    -- 1. PCID 별 대표 userid 부여하는 로직 (해당 기간동안 계속 비회원이면 비회원, 해당기간동안 한번이라도 로그인하면 최초 로그인 한 userid 로 부여)
    SELECT
        DISTINCT PCID,
        FIRST_VALUE(USERID) IGNORE NULLS OVER (PARTITION BY PCID ORDER BY BASE_DATE) AS pcid_userid     -- user id 는 이거로 쓰면 됨
        -- ROW_NUMBER() OVER (PARTITION BY PCID ORDER BY BASE_DATE) AS rn,
    FROM
        ENCAR.LOGS_MART.USER_MASTER_ALL
    WHERE
        1=1
        AND base_date BETWEEN DATE('{start_date}') AND DATE('{end_date}')
),
        -- AND PCID = '00008A2F79A64BC2842177950D43FDF8' -> 아주 좋은 예시.

user AS (           -- 1-1.
    SELECT
        ID,             -- 비회원용 부여 id = 'qlry'
        USER_STATUS,    -- 회원/탈퇴/휴면/이용정지/삭제 등등...
        GRADE,
        -- 딜러 / 유저 / 비회원 구분하는 로직
        CASE 
            WHEN GRADE IN ('0', '2')                THEN 'dealer'
            WHEN GRADE = '1' AND ID != 'qlry'       THEN 'member'
            WHEN GRADE = '1' AND ID = 'qlry'        THEN 'non_member'
            ELSE 'Out' END AS user_type,
        
        -- 비회원일 경우 join_dt 삭제하는 로직
        CASE
            WHEN GRADE = '1' AND ID = 'qlry'        THEN NULL
            ELSE JOIN_DT END AS JOIN_DT,
    FROM
        ENCAR.LOGS_MART.ENCAR_USER
    WHERE
        1=1
),

user_base AS (      -- 1-3. 최종 user 정보 정리 테이블
    SELECT
        t1.PCID,
        t1.pcid_userid,
        t2.JOIN_DT,
        CASE
            WHEN t2.user_type IS NOT NULL THEN t2.user_type
            WHEN t2.user_type IS NULL THEN 'non_member' END AS user_type,
    FROM
        user_master AS t1
        LEFT JOIN user AS t2 ON t1.pcid_userid = t2.id
    WHERE
        1=1
),

-- 최종 설명 1. PCID 별 user id 부여 및 딜러/유저 구분

external_user AS (  -- 2. MKT 관련 external 여부 확인하는 테이블 조건은 해당 기간동안의 가장 첫 SOURCE 값이 external / organic / internal 중 해당 하는 값으로 분류
    SELECT
        DISTINCT PCID,
        FIRST_VALUE(SOURCE) IGNORE NULLS OVER (PARTITION BY PCID ORDER BY BASE_DATE) AS FIRST_SOURCE     -- user id 는 이거로 쓰면 됨
    FROM
        ENCAR.LOGS_MART.USER_ATTRIBUTION_PERIOD
    WHERE
        1=1
        AND PERIOD_TYPE = 'd10'                                 -- 외부유입 기여 10일
        AND SOURCE = 'external'
        AND BASE_DATE BETWEEN DATE('{start_date}') AND DATE('{end_date}')
),

user_final AS (     -- 3. (1+2) 최종 테이블 - PCID <- userid, usertype, external 여부 붙인 최종 테이블
    SELECT
        t1.PCID,
        t1.pcid_userid,
        t1.user_type,
        t1.JOIN_DT,
        CASE
            WHEN t2.FIRST_SOURCE IS NOT NULL THEN t2.FIRST_SOURCE
            ELSE 'organic' END AS SOURCE                                -- external 이 아닌 PCID 는 모두 organic
    FROM
        user_base AS t1
        LEFT JOIN external_user AS t2 ON t2.PCID = t1.PCID
),

-- user_base = external_user 의 총 PCID 카운트는 정확히 동일 너무 좋다.


log_base AS (                   -- 1. log 원천
    SELECT
        t1.BASE_DATE,
        t1.EVENTTIME,
        t1.PCID,
        t1.SCREENNAME,
        t1.EVENTTYPE,
        t1.EVENTNAME,
        t1.EVENTNAMEGROUP,
        t1.STATUS,
        t1.BEF_SCREENNAME,
        t1.BEF_BOARD,
        t1.BEF_EVENTNAMEGROUP,
        t1.HIT,
        t1.ATTRIBUTES,
        t1.ESTIMATEID,
        t1.OS_TYPE,
        t1.OS_DETAIL,
        
        t2.pcid_userid,
        t2.user_type,
        t2.JOIN_DT,
        t2.SOURCE,
        
        
    FROM
        ENCAR.ANALYSIS_MART.ESTIMATE_SESSION_STAGE AS t1
        LEFT JOIN user_final AS t2 ON t2.PCID = t1.PCID
    WHERE
        1=1
        AND BASE_DATE BETWEEN DATE('{start_date}') AND DATE('{end_date}')
),

log_with_flags AS (             -- 2. log 원천 -> row별 특정 액션 여부 1,0 부여
    SELECT 
        PCID,
        SOURCE,
        USER_TYPE,
        OS_TYPE,
        OS_DETAIL,
        
        -- 이벤트 1.        내차팔기 홈
        MAX(CASE 
            WHEN EVENTTYPE = 'VIEW' AND SCREENNAME = '내차팔기_엔카홈'
            THEN 1 ELSE 0 END) AS is_sell_home,

        -- 이벤트 2. 배너 -> 내차팔기 홈
        MAX(CASE 
            WHEN EVENTTYPE = 'VIEW' AND SCREENNAME = '내차팔기_엔카홈' AND HIT = 'indextop'     --  → 상단배너 클릭으로 들어온 진입한 경우엔 hit값이 indextop 으로 찍힘
            THEN 1 ELSE 0 END) AS is_banner_to_sell_home,

        -- 이벤트 3. 내차고 등록 완료 여부
        MAX(CASE 
            WHEN EVENTTYPE = 'VIEW' AND SCREENNAME = '내차고_엔카홈'
                 AND BEF_SCREENNAME IN (
                    '비교견적신청_정보확인중','비교견적신청_커피이벤트','비교견적신청_제조사수기선택','비교견적신청_화물제조사',
                    '비교견적신청_세부등급','비교견적신청_등급','비교견적신청_화물적재용량','비교견적신청_변속기','비교견적신청_연료',
                    '비교견적신청_화물변속기','비교견적신청_기타옵션','비교견적신청_화물등급','비교견적신청_형식연도수기선택',
                    '비교견적신청_화물모델연도','비교견적신청_화물모델','비교견적신청_화물세부형식','비교견적신청_화물승용선택',
                    '비교견적신청_화물형식','비교견적신청_등급수기선택','비교견적신청_리스','비교견적신청_세부모델수기선택',
                    '비교견적신청_세부등급수기선택','비교견적신청_모델수기선택','비교견적신청_화물연료','비교견적신청_모델',
                    '비교견적신청_연식수기선택','비교견적신청_화물적재규격','비교견적신청_화물연식'
                 )
            THEN 1 ELSE 0 END) AS is_mygarage_complete,

        -- 이벤트 4. 판매준비하기 진입 여부
        MAX(CASE
            WHEN EVENTTYPE = 'VIEW' AND SCREENNAME = '비교견적판매준비하기' THEN 1 ELSE 0 END) AS is_ready_to_sell,

        -- 이벤트 5. 빅교견적 신청 완료 여부
        MAX(CASE 
            WHEN EVENTTYPE = 'VIEW' AND SCREENNAME IN ('비교견적_신청완료', '비교견적신청_프로신청완료', '비교견적플러스신청_신청완료')
            THEN 1 ELSE 0 END) AS is_estimate_complete,
    FROM log_base
    GROUP BY
        ALL
),

prep AS (
    SELECT
        SOURCE,
        COUNT(DISTINCT PCID) AS CNT_PCID,
        COUNT(DISTINCT CASE WHEN is_sell_home = 1 THEN PCID END) AS SELL_HOME,
        COUNT(DISTINCT CASE WHEN is_sell_home = 1 AND is_mygarage_complete = 1 THEN PCID END) AS MYGARAGE_COMPLETE,
        COUNT(DISTINCT CASE WHEN is_sell_home = 1 AND is_mygarage_complete = 1 AND is_estimate_complete = 1 THEN PCID END) AS ESTIMATE_COMPLETE,
        -- COUNT(DISTINCT CASE WHEN is_mygarage_complete = 1 AND is_ready_to_sell = 1 THEN PCID END) AS READY_TO_SELL,
        
        -- 배너 클릭을 통해 내차팔기 홈 진입한 유저
        COUNT(DISTINCT CASE WHEN is_banner_to_sell_home = 1 THEN PCID END) AS BANNER_TO_SELL_HOME,
        COUNT(DISTINCT CASE WHEN is_banner_to_sell_home = 1 AND is_mygarage_complete = 1 THEN PCID END) AS BANNER_TO_MYGARAGE_COMPLETE,
        COUNT(DISTINCT CASE WHEN is_banner_to_sell_home = 1 AND is_mygarage_complete = 1 AND is_estimate_complete = 1 THEN PCID END) AS BANNER_TO_ESTIMATE_COMPLETE,
        -- COUNT(DISTINCT CASE WHEN is_banner_to_sell_home = 1 AND is_mygarage_complete = 1 AND is_ready_to_sell = 1 THEN PCID END) AS BANNER_TO_READY_TO_SELL,
    FROM
        log_with_flags
    WHERE
        1=1
        AND user_type NOT IN ('Out', 'dealer')        -- 탈퇴 및 딜러 유저 제외
    GROUP BY 
        GROUPING SETS ((SOURCE), ())
)

SELECT
    DATE('{start_date}'),
    DATE('{end_date}'),
    *
FROM
    prep
ORDER BY
    SOURCE
;
'''
df = pd.read_sql(query, engine)

# 결과 확인
df

,DATE('2025-07-22'),DATE('2025-07-29'),source,cnt_pcid,sell_home,mygarage_complete,estimate_complete,banner_to_sell_home,banner_to_mygarage_complete,banner_to_estimate_complete
0,2025-07-22,2025-07-29,external,7047,6260,2310,176,2735,786,49
1,2025-07-22,2025-07-29,organic,298919,272452,29740,942,192258,13309,226
2,2025-07-22,2025-07-29,None,305966,278712,32050,1118,194993,14095,275


### sql 결과 상세 설명

- source = None -> external, organic 전부 포함
- banner_to_~~ -> 상단 배너 클릭을 통해 내차팔기 홈에 진입한 여부가 있고 다음 액션 유무 여부 

====================================================================================================================================================================================

# 분석 위키

# 1. 배경

- 인덱스 상단 배너를 통한 내차등록 효율이 높아짐에 따라, 일 차량 등록 수가 2배 가량 높아짐  
- 이에 따라, **Paid Marketing**을 통해 차량등록을 시키는 데 소진하는 비용을 감소시킬 수 있을지 판단하고자 함  
- **가설:**  
  - Paid 유저들의 차량등록-견적신청 전환율이  
    인덱스상단배너를 통해 유입된 오가닉 유저들의 차량등록-견적신청 전환율과 **동일하거나 낮을 것**  
  - 그렇다면 **Paid Marketing의 비용을 조정하고 오가닉 전환에 더 집중**하는 것이 효율적임

---

# 2. 분석 요청사항 상세

> 요청 원문

1. **Paid User**  
   - 10일 내 Paid 광고를 통해 유입되어 등록완료한 고객들의 모수  
   - 이들 중 **비교견적 신청 완료 수와 비율**  
   - (상단 배너를 거쳐 등록했을 수도 있음)

2. **Banner User**  
   - 인덱스 상단 배너를 통해 차량등록을 한 고객들의 모수  
   - 이들 중 **비교견적 신청 완료 수와 비율**  
   - (Paid 유입 고객 포함 가능)

3. **Pure-organic User**  
   - **Paid가 아닌 Organic 유저 중**, 상단배너를 통해 차량등록을 한 고객들의 모수  
   - 이들 중 **비교견적 신청 완료 수와 비율**

- **분석 대상 기간**: `2025.07.22 ~ 2025.07.29`  
- **대상자**: 해당 기간 중 차량등록을 진행한 고객 중 전환율 분석

---

## 2-1. Funnel 정의 요약

| 구분          | 단계1                    | 단계2                                    | 단계3                                              |
|---------------|--------------------------|-------------------------------------------|-----------------------------------------------------|
| Paid User     | 내차고 등록 완료         | → 비교견적 신청 완료                     |                                                     |
| Banner User   | 상단배너 → 내차팔기 홈 진입 | → 내차고 등록 완료                        | → 비교견적 신청 완료                                 |
| Organic User  | 상단배너 → 내차팔기 홈 진입 | → 내차고 등록 완료                        | → 비교견적 신청 완료                                 |

---

# 3. 분석 Insight

- 마케팅 캠페인으로 유입된 **Paid 유저(external)** 는 모수가 작지만, **내차팔기 intent가 높은 유저**로서 **오가닉-배너 유입 유저보다 비교견적 신청 전환율이 높음**

---

# 4. 분석 상세 내용

![분석 차트](../etc/image1.png)



## 4-0. 분석 조건

- **분석 기간**: `2025.07.22 ~ 2025.07.29`  
- **External (Paid) 조건**:  
  - 마케팅 기여 10일 내  
  - `ENCAR.LOGS_MART.USER_ATTRIBUTION_PERIOD` 테이블 기준  
  - `PERIOD_TYPE = 'd10' AND SOURCE = 'external'` → Paid  
  - 나머지 PCID는 모두 Organic으로 간주
- **User type 조건**:  
  - 딜러 및 탈퇴 회원 제외 (`user_master_all` 테이블)  
  - 딜러는 비교견적 신청 완료를 거의 하지 않음 (전환율 0.02%p 영향 미미)
- **유저 분류 시점**:  
  - 해당 기간 내 **첫 방문 날짜** 기준으로 external 및 user type 판별
- **퍼널 조건**:  
  - 앞 단계 액션이 존재할 때만 다음 단계 전환 인정  
  - 예: 비교견적 신청 완료 → 내차팔기 홈 진입 + 내차고 신규등록 조건 만족 시 인정

---

## 4-1. 내차팔기 홈 신규유입 모수 및 전환율 비교 (`초록색 / 빨간색 영역`)

- **external 유저**의 내차팔기 홈(신규유입) 모수는 **오가닉-배너 유저 대비 3% 수준**  
  - external: 약 6,000명  
  - organic-banner: 약 190,000명
- 하지만, 전환율은 external 유저가 **2배 이상 높음**  
  - 비교견적 신청 전환율:  
    - external: **7.6%**  
    - organic-banner: **1.7%** (→ 약 **4.5배 차이**)
- 배너 유입은 전체 전환 기여에서 각각:
  - Paid 유저의 전환 중 **28%**,  
  - Organic 유저의 전환 중 **24%**를 차지  
→ **배너는 두 그룹 모두에 중요한 유입 채널**

---

## 4-2. external vs organic 유저의 배너 이용 비중 (`파란색 영역`)

- 내차팔기 홈(신규유입) 진입에서 배너 비중:
  - external 유입: **40%**
  - organic 유입: **71%**
- 최종 전환(비교견적 신청) 기준 배너 기여 비율:
  - external: **28%**
  - organic: **24%**

→ **배너는 모든 유입 경로에서 전환 기여에 유의미한 영향**을 주고 있음